# Calculating inputs for Dynamic Benthic Pelagic Model (DBPM)
**Author:** Denisse Fierro Arcos  
**Date:** 2024-10-14  
  
DBPM requires inputs that are not directly available as GFDL outputs, but these can be derived from GFDL data. We will calculate the following variables:  
- `lphy` (large phytoplankton) - This is the difference between  `phyc_vint` and small phytoplankton (i.e., `phypico_vint`)  
- `er` (export ratio) - This is calculated from small (`phypico_vint`) and large phytoplankton (`lphy`), sea surface temperature (`tos`) and depth (`deptho`) using the `getExportRatio` function
- `intercept` - This is calculated from small 

In [2]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/python_workflow')

## Loading relevant libraries

In [3]:
import xarray as xr
import pandas as pd
import numpy as np
from glob import glob
from dask.distributed import Client
import useful_functions as uf

## Start a cluster

In [ ]:
client = Client(threads_per_worker = 1)
client

## Calculating phytoplankton (large and small) and export ratio

In [3]:
#Define output folder
out_folder = '/g/data/vf71/la6889/dbpm_inputs/processed_forcings_gridded'
os.makedirs(out_folder, exist_ok = True)

#Define location of folder containing GFDL outputs
gridded_data_folder = '/g/data/vf71/la6889/dbpm_inputs/gridded/'

In [11]:
#Calculating from ctrlclim data
gfdl_exp = 'ctrlclim'
sphy_ctrl, lphy_ctrl, er_ctrl = uf.getExportRatio(gridded_data_folder, gfdl_exp)

In [12]:
#Saving datasets
sphy_ctrl.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_sphy_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

lphy_ctrl.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_lphy_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

er_ctrl.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_er_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

In [13]:
#Calculating from obsclim data
gfdl_exp = 'obsclim'
sphy_obs, lphy_obs, er_obs = uf.getExportRatio(gridded_data_folder, gfdl_exp)

In [14]:
#Saving datasets
sphy_obs.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_sphy_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

lphy_obs.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_lphy_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

er_obs.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_er_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

## Calculating intercept and slope
Method to calculate intercept from appendix of Barnes et al. 2010 JPR.

We need depth, and phytoplankton (small and large) to calculate the slope and intercept using the `GetPPIntSlope` function.

In [20]:
#Loading ctrlclim data
gfdl_exp = 'ctrlclim'
sphy = xr.open_zarr(
    glob(os.path.join(out_folder, f'*{gfdl_exp}_sphy*'))[0])['sphy']

lphy = xr.open_zarr(
    glob(os.path.join(out_folder, f'*{gfdl_exp}_lphy*'))[0])['lphy']

depth = xr.open_zarr(
    glob(os.path.join('/g/data/vf71/la6889/dbpm_inputs/gridded/', 
                      f'*{gfdl_exp}_deptho*'))[0])['deptho']

#Calculating intercept and slope
intercept, slope = uf.GetPPIntSlope(sphy, lphy, depth)

### Saving `ctrlclim` results

In [8]:
#Saving datasets
intercept.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_intercept_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

slope.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_slope_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

In [21]:
#Loading obsclim data
gfdl_exp = 'obsclim'
sphy = xr.open_zarr(
    glob(os.path.join(out_folder, f'*{gfdl_exp}_sphy*'))[0])['sphy']

lphy = xr.open_zarr(
    glob(os.path.join(out_folder, f'*{gfdl_exp}_lphy*'))[0])['lphy']

depth = xr.open_zarr(
    glob(os.path.join('/g/data/vf71/la6889/dbpm_inputs/gridded/', 
                      f'*{gfdl_exp}_deptho*'))[0])['deptho']

#Calculating intercept and slope
intercept, slope = uf.GetPPIntSlope(sphy, lphy, depth)

### Saving `obsclim` results

In [23]:
#Saving datasets
intercept.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_intercept_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

slope.to_zarr(
    os.path.join(out_folder, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_slope_15arcmin_lme-61_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

## Creating **spinup** period
We will use data from `ctrlclim` between 1961 and 1980 as our spinup period. Our spinup period will go from 1841 to 1960.

In [14]:
spin_files = glob(os.path.join(out_folder, '*ctrlclim*'))
spinup_period = pd.date_range('1841-01-01', end = '1960-12-31', freq = 'MS')

In [15]:
for f in spin_files:
    f_out = f.replace('ctrlclim', 'spinup')
    uf.gridded_spinup(f, '1961', '1980', spinup_period, file_out = f_out)

In [16]:
#Variables of interest
dbpm_var = ['expc-bot', 'tob', 'tos']
gfdl_gridded = '/g/data/vf71/la6889/dbpm_inputs/gridded'
spin_files = [glob(os.path.join(gfdl_gridded, f'*ctrlclim_{v}*'))[0]for v in dbpm_var]

In [23]:
for f in spin_files:
    f_out = os.path.basename(f).replace('ctrlclim', 'spinup')
    uf.gridded_spinup(spin_files[0], 1961, 1980, spinup_period, 
                      file_out = os.path.join(out_folder, f_out))

In [4]:
df = pd.read_parquet('/g/data/vf71/la6889/dbpm_inputs/monthly_weighted_mean/ctrlclim_dbpm_all-inputs_LME_61_1961-2010.parquet')
df.head()

,region,scenario,time,year,month,depth_m,tot_area_m2,expc-bot_mol_m-2_s-1,phyc-vint_mol_m-2,phypico-vint_mol_m-2,tob_degC,tos_degC
0,LME 61,ctrlclim,1961-01-01,1961,January,519.051326,2.639106e+12,1.814283e-07,0.322527,0.082182,-1.335118,0.330517
1,LME 61,ctrlclim,1961-02-01,1961,February,519.051326,2.639106e+12,7.058071e-08,0.228363,0.118517,-1.291303,0.130712
2,LME 61,ctrlclim,1961-03-01,1961,March,519.051326,2.639106e+12,3.594295e-08,0.163950,0.101095,-1.279956,-0.971242
3,LME 61,ctrlclim,1961-04-01,1961,April,519.051326,2.639106e+12,1.602367e-08,0.078121,0.048714,-1.303288,-1.593834
4,LME 61,ctrlclim,1961-05-01,1961,May,519.051326,2.639106e+12,7.000150e-09,0.027814,0.018355,-1.329981,-1.760516


In [30]:
test = slope.isel(time = 0).to_pandas().reset_index().melt(id_vars = 'lat').dropna()
test.sort_values('lat').sort_values('lon')

,lat,lon,value
67,-77.375,-179.875,-0.842528
64,-76.625,-179.875,-0.810762
68,-77.625,-179.875,-0.910071
53,-73.875,-179.875,-1.159463
57,-74.875,-179.875,-1.018375
...,...,...,...
143964,-76.625,179.875,-0.806387
143961,-75.875,179.875,-0.843549
143959,-75.375,179.875,-0.835005
143965,-76.875,179.875,-0.860093


In [69]:
# x = pd.read_csv('/g/data/vf71/fishmip_inputs/ISIMIP3a/processed_forcings/lme_inputs_gridcell/obsclim/025deg/obsclim_historical_LME_61_all_inputs.csv')
x.head()

,lat,lon,region,t,sst,sbt,er,intercept,slope,sphy,lphy,depth,area_m2,expcbot,year,month,scenario
0,-78.375,-169.375,LME_61,1841-01-01,-0.538,-1.428,0.215,1.526,-0.958,0.10162,0.16367,377.324524,1.557166e+08,1.221457e-07,1841,Jan,spinup
1,-78.375,-169.125,LME_61,1841-01-01,-0.429,-1.457,0.216,1.683,-0.954,0.13111,0.22203,382.192932,1.557166e+08,1.342319e-07,1841,Jan,spinup
2,-78.375,-168.875,LME_61,1841-01-01,-0.203,-1.439,0.211,1.632,-0.956,0.12424,0.20418,386.661469,1.557166e+08,1.412280e-07,1841,Jan,spinup
3,-78.375,-168.625,LME_61,1841-01-01,-0.101,-1.434,0.209,1.620,-0.957,0.12264,0.20011,392.076996,1.557166e+08,1.472865e-07,1841,Jan,spinup
4,-78.375,-168.375,LME_61,1841-01-01,-0.049,-1.427,0.208,1.625,-0.956,0.12146,0.20012,394.806458,1.557166e+08,1.521465e-07,1841,Jan,spinup


In [135]:
x[x.t == '1841-01-01'].sort_values('lat').sort_values('lon')

,lat,lon,region,t,sst,sbt,er,intercept,slope,sphy,lphy,depth,area_m2,expcbot,year,month,scenario
4216,-73.625,-179.875,LME_61,1841-01-01,-1.261,-1.376,0.109,0.546,-1.133,0.73448,0.16122,356.848389,2.178606e+08,1.611782e-07,1841,Jan,spinup
2208,-75.625,-179.875,LME_61,1841-01-01,0.444,-1.843,0.212,2.454,-0.829,0.03831,0.26686,576.998291,1.918526e+08,3.010871e-07,1841,Jan,spinup
617,-77.125,-179.875,LME_61,1841-01-01,0.452,-1.864,0.173,2.312,-0.861,0.05979,0.28957,740.681519,1.721915e+08,1.633900e-07,1841,Jan,spinup
1122,-76.625,-179.875,LME_61,1841-01-01,0.178,-1.861,0.216,2.612,-0.811,0.03538,0.30367,586.550842,1.787590e+08,1.908097e-07,1841,Jan,spinup
4967,-72.875,-179.875,LME_61,1841-01-01,-0.051,-0.401,0.140,2.419,-0.890,0.15364,0.53596,954.519104,2.275468e+08,1.233114e-07,1841,Jan,spinup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3713,-74.375,179.875,LME_61,1841-01-01,-1.282,-1.373,0.153,0.742,-1.104,0.57126,0.17450,235.570419,2.081370e+08,1.990710e-07,1841,Jan,spinup
852,-77.125,179.875,LME_61,1841-01-01,0.765,-1.863,0.162,2.130,-0.890,0.07971,0.27687,743.643066,1.721915e+08,1.641296e-07,1841,Jan,spinup
5213,-72.875,179.875,LME_61,1841-01-01,0.007,-0.168,0.121,2.492,-0.874,0.12244,0.51444,1221.237305,2.275468e+08,9.100065e-08,1841,Jan,spinup
4966,-73.125,179.875,LME_61,1841-01-01,-0.047,-0.825,0.164,1.915,-0.969,0.32684,0.46289,544.887329,2.243223e+08,2.015334e-07,1841,Jan,spinup
